In [1]:
import findspark
findspark.init("/opt/manual/spark/")

In [2]:
from pyspark.sql import SparkSession, functions as F

In [3]:
# Note the master is yarn and enableHiveSupport()

spark = SparkSession.builder \
.appName("Spark SQL") \
.master("local[2]") \
.enableHiveSupport() \
.getOrCreate()

2022-08-27 20:07:38,323 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
! wget -O ~/datasets/Churn_Modelling.csv \
https://github.com/erkansirin78/datasets/raw/master/Churn_Modelling.csv

--2022-08-27 20:09:43--  https://github.com/erkansirin78/datasets/raw/master/Churn_Modelling.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/Churn_Modelling.csv [following]
--2022-08-27 20:09:44--  https://raw.githubusercontent.com/erkansirin78/datasets/master/Churn_Modelling.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 674856 (659K) [text/plain]
Saving to: ‘/home/train/datasets/Churn_Modelling.csv’

100%[======================================>] 674,856     2.27MB/s   in 0.3s   

2022-08-27 20:09:44 (2.27 MB/s) - ‘/home/train/datasets/Churn_Modelling.csv’ 

In [7]:
! ls -l ~/datasets | grep Churn

-rw-rw-r--. 1 train train   674856 Aug 27 20:09 Churn_Modelling.csv


In [8]:
df = spark.read.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("sep", ",") \
.load("file:///home/train/datasets/Churn_Modelling.csv")

In [9]:
df.limit(5).toPandas()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
df.createOrReplaceTempView("churn_temp")

In [12]:
df_geo = spark.sql("""
    
    
    SELECT Geography, COUNT(RowNumber) 
    FROM churn_temp 
    GROUP BY Geography
    LIMIT 5;



""")

In [13]:
df_geo.show()

+---------+----------------+
|Geography|count(RowNumber)|
+---------+----------------+
|  Germany|            2509|
|   France|            5014|
|    Spain|            2477|
+---------+----------------+



In [14]:
spark.stop()